# Import Libraries
Libraries yang dibutuhkan untuk melakukan Clustering

In [350]:
import pandas as pd 
import numpy as np

#library untuk melakukan pengolahan file Excel
!pip install openpyxl

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
%matplotlib inline
from scipy import stats

# Data Understanding
Dataset yang digunakan dalam Clustering ini adalah data rasio keuangan dari Industri Hotel, Restoran, dan Pariwisata tahun 2021 yang tercatat di Bursa Efek Indonesia.

3 variabel yang digunakan antara lain :
* **Debt to Equity ratio (DER)**, 
* **Return on Asset (ROA)**, dan 
* **Sales Growth.** 

Tahap pra-pengolahan data dilakukan dengan menggunakan Ms. Excel.

> # 1. Exploratory Data Analysis
> > # 1.1 Load Data
Melakukan pembacaan data Excel

In [351]:
df0 = pd.read_excel('../input/industri/data-emiten.xlsx')
df0.info()

> > # 1.2 Memeriksa nilai null

In [352]:
print(df0.isna().sum())

> > # 1.3 Mengganti nilai null dengan 0

In [353]:
df0.fillna(0, inplace=True)

> > # 1.4 Deskripsi Data

In [354]:
df0.describe()

> # 1.5 Data Cleansing
> Drop kolom yang tidak dipakai untuk penentuan Scatter Plot

In [355]:
df0.drop(['Category'], axis=1, inplace=True)

> # 1.6 Scatter Plot

In [356]:
plt.figure(figsize=(15, 15))
sns.pairplot(df0, vars = df0.columns[1:13]);

# 2. Clustering
> > # 2.1 Load Data

In [357]:
df0 = pd.read_excel('../input/industri/data-emiten.xlsx')
df1 = df0[['DER (X)','Return On Asset','Sales Growth','Category']]
df1.describe()

In [358]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='Return On Asset', 
                data=df0, s=80)

In [359]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='DER (X)', 
                data=df0, s=80)

In [360]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='Sales Growth', 
                data=df0, s=80)

> > # 2.2 Outlier
> Menghapus Outlier menggunakan z-score

In [361]:
print(f'Jumlah baris sebelum memfilter outlier: {len(df1)}')

filtered_entries = np.array([False] * len(df1))
for col in ['Return On Asset']:
    zscore = abs(stats.zscore(df1[col]))
    filtered_entries = (zscore < 3) | filtered_entries
    
df1 = df1[filtered_entries]

print(f'Jumlah baris setelah memfilter outlier: {len(df1)}')

In [362]:
df1.describe()

> > # 2.3 Standardisasi Fitur
> Melakukan normalisasi data menggunakan Standard Scaler

In [363]:
feats = ['DER (X)','Return On Asset','Sales Growth']
X = df1[feats].values
y = df1['Category'].values

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)
new_df = pd.DataFrame(data = X_std, columns = feats)
new_df.describe()

> > # 2.4 Mencari nilai k (elbow point)
> Mencari berapakah nilai k (elbow point) yang paling optimal untuk melakukan Clustering

In [364]:
from sklearn.cluster import KMeans
inertia = []

for i in range(1, 32):
  kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
  kmeans.fit(new_df.values)
  inertia.append(kmeans.inertia_)

plt.figure(figsize=(12, 6))
plt.plot(inertia) #Elbow Method

> > Ternyata clustering paling optimal di 5 cluster (n_cluster = 5)

> > # 2.5 Lakukan K-means, k = 5

In [365]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(new_df.values)

In [366]:
new_df['target'] = y
new_df['fit'] = kmeans.labels_

> > # 2.6 Plot Cluster

In [367]:
plt.rcParams["figure.figsize"] = (8, 8)
sns.scatterplot(x='DER (X)', y='Return On Asset', hue=new_df.target.to_list(), data=new_df);

In [368]:
new_df.head(5)

> > Mencocokan nilai fit dengan Category untuk membuat indikator warna yang benar. 

In [369]:
new_df.insert(0,"Category",np.nan,True)
new_df.loc[new_df['fit'] == 0, 'Category'] = 2
new_df.loc[new_df['fit'] == 1, 'Category'] = 4
new_df.loc[new_df['fit'] == 2, 'Category'] = 1
new_df.loc[new_df['fit'] == 3, 'Category'] = 0
new_df.loc[new_df['fit'] == 4, 'Category'] = 3
new_df['Category'] = new_df['Category'].astype(int)
new_df.head(5)

> > Menyesuaikan nama kolom menjadi (St.) = Standardized

In [370]:
new_df.drop(["fit","target"], axis=1, inplace=True)
new_df.columns = ['Category','DER (St.)','Return On Asset (St.)', 'Sales Growth (St.)']
new_df.info()

> > > # 2.6.1 3D Scatter Plot

In [371]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


x = new_df['DER (St.)']
y = new_df['Return On Asset (St.)']
z = new_df['Sales Growth (St.)']

ax.set_xlabel("DER (St.)")
ax.set_ylabel("Return On Asset (St.)")
ax.set_zlabel("Sales Growth (St.)")

cmap = ListedColormap(sns.color_palette("RdYlGn", 256).as_hex())

sc = ax.scatter(x, y, z, s=40, c=new_df['Category'], marker='o', cmap=cmap, alpha=1)
plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)

plt.show()

> > > # 2.6.2 2D Scatter Plot (X vs Z View)

In [372]:
# color palette as dictionary
palette = {0:"tab:red",
           1:"tab:orange", 
           2:"yellow",
           3:"lime",
           4:"darkgreen"}

sns.scatterplot(x='DER (St.)', y='Sales Growth (St.)', 
                hue='Category', data=new_df, s=80,
                palette=palette)
plt.title("DER vs Sales Growth (Standardisasi)", fontsize=18, y=1.05)
plt.show()

> > > # 2.6.3 2D Scatter Plot (Y vs Z View)

In [373]:
sns.scatterplot(x='Return On Asset (St.)', y='Sales Growth (St.)', 
                hue='Category', data=new_df, s=80,
                palette=palette)
plt.title("Return On Asset vs Sales Growth (Standardisasi)", fontsize=18, y=1.05)
plt.show()

> > > # 2.6.4 2D Scatter Plot (X vs Y View)

In [374]:
sns.scatterplot(x='DER (St.)', y='Return On Asset (St.)', 
                hue='Category', data=new_df, s=80,
                palette=palette)
plt.title("DER vs Return On Asset (Standardisasi)", fontsize=18, y=1.05)
plt.show()

# 3. Export Result
Drop kolom yang tidak digunakan pada dataset Awal **(df0).**

In [375]:
df0.drop(['Category'], axis=1, inplace=True)

Gabungkan dataset List Emiten Pariwisata **(dftemp)**, Result **(new_df)**, dan dataset Awal **(df0)** ke dalam satu dataframe. 

In [376]:
dftemp = pd.read_excel('../input/industri/pariwisata-list.xlsx')
df = pd.concat([dftemp, new_df, df0], axis=1)
df.head(5)

> # 3.1 Export hasil ke Excel

In [377]:
# membuat objek excel writer
writer = pd.ExcelWriter('output.xlsx')

# menulis dataframe ke Excel
df.to_excel(writer)

# menyimpan file Excel
writer.save()
print('DataFrame is written successfully to Excel File.')

# 4. Get Centroid
> # 4.1 Jumlah populasi dari setiap Centroid

In [378]:
clusters = kmeans.fit_predict(X_std)
unique_elements, counts_elements = np.unique(clusters, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

> # 4.2 Letak Centroid pada setiap Cluster

In [379]:
kmeans.cluster_centers_

# 5. Cluster Description
> # 5.1 Cluster yang terdampak Sangat Negatif

In [380]:
#Highly Negatively Impacted
df[df['Category'] == 4].shape

In [381]:
display(df[df['Category'] == 4])

> # 5.2 Cluster yang terdampak Negatif

In [382]:
#Negatively Impacted
df[df['Category'] == 3].shape

In [383]:
display(df[df['Category'] == 3])

> # 5.3 Cluster yang tidak terdampak (Netral)

In [384]:
#Neutral Impacted
df[df['Category'] == 2].shape

In [385]:
display(df[df['Category'] == 2])

> # 5.4 Cluster yang terdampak Positif

In [386]:
#Positively Impacted
df[df['Category'] == 1].shape

In [387]:
display(df[df['Category'] == 1])

> # 5.5 Cluster yang terdampak Sangat Positif

In [388]:
#Highly Positively Impacted
df[df['Category'] == 0].shape

In [389]:
display(df[df['Category'] == 0])